In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np
import pandas as pd
import lightgbm
import os
os.chdir('/Users/hanbosun/Documents/GitHub/TrasactionPrediction/')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer



/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:

#
# Prepare the data
#
train = pd.read_csv('example/train.csv')
train = train.iloc[1:5000,:]

# get the labels
y = train.target.values
train.drop(['id', 'target'], inplace=True, axis=1)

x = train.values


In [3]:

#
# Create training and validation sets
#
x, x_test, y, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

#
# Create the LightGBM data containers
#
categorical_features = [c for c, col in enumerate(train.columns) if 'cat' in col]
train_data = lightgbm.Dataset(x, label=y, categorical_feature=categorical_features)
test_data = lightgbm.Dataset(x_test, label=y_test)


#
# Train the model
#

parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}

model = lightgbm.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=5000,
                       early_stopping_rounds=100)
#
# Create a submission
#

submission = pd.read_csv('example/test.csv')
ids = submission['id'].values
submission.drop('id', inplace=True, axis=1)


x = submission.values
y = model.predict(x)

output = pd.DataFrame({'id': ids, 'target': y})
output.to_csv("submission.csv", index=False)

/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1190: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [1, 3, 4, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:752: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[1]	valid_0's auc: 0.526987
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.512879
[3]	valid_0's auc: 0.520355
[4]	valid_0's auc: 0.534167
[5]	valid_0's auc: 0.524619
[6]	valid_0's auc: 0.523568
[7]	valid_0's auc: 0.537513
[8]	valid_0's auc: 0.538845
[9]	valid_0's auc: 0.535707
[10]	valid_0's auc: 0.533605
[11]	valid_0's auc: 0.529933
[12]	valid_0's auc: 0.527476
[13]	valid_0's auc: 0.525315
[14]	valid_0's auc: 0.520696
[15]	valid_0's auc: 0.524693
[16]	valid_0's auc: 0.527905
[17]	valid_0's auc: 0.52456
[18]	valid_0's auc: 0.519704
[19]	valid_0's auc: 0.522902
[20]	valid_0's auc: 0.517365
[21]	valid_0's auc: 0.521303
[22]	valid_0's auc: 0.525063
[23]	valid_0's auc: 0.523346
[24]	valid_0's auc: 0.517898
[25]	valid_0's auc: 0.518993
[26]	valid_0's auc: 0.528764
[27]	valid_0's auc: 0.536166
[28]	valid_0's auc: 0.535959
[29]	valid_0's auc: 0.535662
[30]	valid_0's auc: 0.534715
[31]	valid_0's auc: 0.537972
[32]	valid_0's auc: 0.538594
[33]	valid_0's auc: